In [1]:
import numpy as np
import pandas as pd
import time
import sys
import os
import shutil

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from dataloader import get_data
from models import TransformerClassifier

In [2]:
ls

Evaluate.ipynb            __pycache__/              dataloader.py
Huggingface Bert.ipynb    bert_dataloader.py        models.py
Model Loading.ipynb       bert_evaluate.py          precomputed/
Multi Head model.ipynb    bert_models.py            train.py
Simple Models.ipynb       bert_train.py
Test dataloader.ipynb     bert_train_multi_head.py


In [3]:
vocab, data_dict = get_data()

In [4]:
wiki_data, fake_data = data_dict['wiki'], data_dict['fake news']

In [5]:
class NaiveBayes(nn.Module):
    def __init__(self, vocab, num_labels, train_data, alpha=0.001):
        super(NaiveBayes, self).__init__()
        self.vocab_len = len(vocab)
        self.classes = num_labels
        self.p_class = np.zeros(self.classes)
        self.p_vocab = alpha * np.ones((self.classes, self.vocab_len))
        for (x, y) in train_data:
            self.p_class[y] += 1
            for i in x:
                if (i == 0): break # 0 padding
                self.p_vocab[y, i] += 1
        self.p_class /= np.sum(self.p_class)
        self.p_vocab = (self.p_vocab.T / np.sum(self.p_vocab, axis=1)).T
        
    def forward(self, src):
        log_probs = np.log(self.p_class)
        for i in src:
            if (i == 0): break
            log_probs += np.log(self.p_vocab[:,i])
        return np.argmax(log_probs)

In [6]:
nb_model = NaiveBayes(vocab, wiki_data.num_labels(), wiki_data)

In [7]:
nb_model([1, 2, 3 ,4])

1

In [8]:
# split dataset into train, validation, and test
def split_dataset(dataset, train_size, val_size, test_size):
    return torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

def evaluate(model, val_dataset):
    n = len(val_dataset)
    pred, true = np.zeros(n, dtype=int), np.zeros(n, dtype=int)
    for i, (x, y) in enumerate(val_dataset):
        true[i] = y
        pred[i] = model(x)
    print("accuracy: ", round(np.mean(pred == true), 4))
    class_scores = [np.mean(pred[true==i] == true[true==i]) for i in range(np.max(true)+1)]
    print("class-wise accuracies: ", class_scores)
    return np.mean(pred == true)

In [20]:
def run_model_on_dataset(model_class, dataset):
    n = len(dataset)
    n_train, n_val, n_test = n - 2*int(0.2*n), int(0.2*n), int(0.2*n)
    train, val, test = split_dataset(dataset, n_train, n_val, n_test)
    best_model, best_score, best_alpha = None, 0.0, 0.0
    for alpha in [0.01, 0.05, 0.1, 0.5, 1, 3, 6, 10]:
        nb_model = model_class(vocab, dataset.num_labels(), train, alpha=alpha)
        print("evaluate model, alpha = ", alpha)
        score = evaluate(nb_model, val)
        if (score > best_score):
            best_model = nb_model
            best_score = score
            best_alpha = alpha
    print("best model alpha={}, evaluating on final test dataset".format(best_alpha))
    return evaluate(nb_model, test)

In [21]:
def print_f1_scores(model_class, dataset, alpha, multi_label=False):
    labels = ['unrelated', 'agree', 'disagree', 'discuss'] if multi_label else ['not attack', 'attack']
    n = len(dataset)
    n_train, n_val, n_test = n - 2*int(0.15*n), int(0.15*n), int(0.15*n)
    train, val, test = split_dataset(dataset, n_train, n_val, n_test)
    nb_model = model_class(vocab, dataset.num_labels(), train, alpha=alpha)
    preds, truth = np.zeros(n, dtype=int), np.zeros(n, dtype=int)
    for i, (x, y) in enumerate(test):
        truth[i] = y
        preds[i] = nb_model(x)
    for i, label in enumerate(labels): # 4 classes
        print("label '{}':".format(label))
        precision = np.mean(preds[preds==i] == truth[preds==i])
        recall = np.mean(preds[truth==i] == truth[truth==i])
        f1_score = 2 * precision * recall / (precision + recall)
        print("f1 score: {:5f}, precision: {:5}, recall: {:5f}".format(f1_score, precision, recall))

In [22]:
run_model_on_dataset(NaiveBayes, wiki_data)

evaluate model, alpha =  0.01
accuracy:  0.8559
class-wise accuracies:  [0.8636910129245566, 0.7988338192419825]
evaluate model, alpha =  0.05
accuracy:  0.8594
class-wise accuracies:  [0.8654443442540828, 0.815597667638484]
evaluate model, alpha =  0.1
accuracy:  0.8741
class-wise accuracies:  [0.88598336839996, 0.7875364431486881]
evaluate model, alpha =  0.5
accuracy:  0.917
class-wise accuracies:  [0.974150886684701, 0.5010932944606414]
evaluate model, alpha =  1
accuracy:  0.9083
class-wise accuracies:  [0.9925859132351468, 0.29518950437317787]
evaluate model, alpha =  3
accuracy:  0.8888
class-wise accuracies:  [0.999499048191564, 0.08381924198250729]
evaluate model, alpha =  6
accuracy:  0.8824
class-wise accuracies:  [0.9997996192766256, 0.02806122448979592]
evaluate model, alpha =  10
accuracy:  0.8811
class-wise accuracies:  [0.9997996192766256, 0.01749271137026239]
best model alpha=0.5, evaluating on final test dataset
accuracy:  0.8868
class-wise accuracies:  [0.99975104560

0.8867700167356646

In [29]:
run_model_on_dataset(NaiveBayes, fake_data)

evaluate model, alpha =  0.01
accuracy:  0.5649
class-wise accuracies:  [0.5230276888827049, 0.561662198391421, 0.6428571428571429, 0.7357268981753973]
evaluate model, alpha =  0.05
accuracy:  0.5841
class-wise accuracies:  [0.5540559343258662, 0.5428954423592494, 0.6011904761904762, 0.7274867569158329]
evaluate model, alpha =  0.1
accuracy:  0.6159
class-wise accuracies:  [0.5978850702657577, 0.532171581769437, 0.5833333333333334, 0.7321954090641554]
evaluate model, alpha =  0.5
accuracy:  0.725
class-wise accuracies:  [0.802560178099346, 0.26273458445040215, 0.047619047619047616, 0.6668628605061802]
evaluate model, alpha =  1
accuracy:  0.7599
class-wise accuracies:  [0.8925838319187421, 0.08310991957104558, 0.0, 0.5709240729841083]
evaluate model, alpha =  3
accuracy:  0.7497
class-wise accuracies:  [0.9980520384026715, 0.0, 0.0, 0.10241318422601531]
evaluate model, alpha =  6
accuracy:  0.7334
class-wise accuracies:  [1.0, 0.0, 0.0, 0.0]
evaluate model, alpha =  10
accuracy:  0.733

0.7295918367346939

In [30]:
print_f1_scores(NaiveBayes, wiki_data, alpha=0.5, multi_label=False) # best alpha=0.5 (see above)

label 'not attack':
f1 score: 0.993880, precision: 0.9923703677232606, recall: 0.995394
label 'attack':
f1 score: 0.613122, precision: 0.6783479349186483, recall: 0.559340


In [31]:
print_f1_scores(NaiveBayes, fake_data, alpha=1.0, multi_label=True) # best alpha=0.5 (see above)

label 'unrelated':
f1 score: 0.982467, precision: 0.9793938625412123, recall: 0.985559
label 'agree':
f1 score: 0.241821, precision: 0.7024793388429752, recall: 0.146048
label 'disagree':
f1 score:   nan, precision:   0.0, recall: 0.000000
label 'discuss':
f1 score: 0.531117, precision: 0.48110185778347214, recall: 0.592739


/Users/johnhallman/mlcourse/mlenv/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


In [32]:
np.mean([0.982467, 0.241821, 0, 0.531117])

0.43885125

In [35]:
# slightly more sophisticated model
class NaiveBayesBigram(nn.Module):
    def __init__(self, vocab, num_labels, train_data, alpha=0.001):
        super(NaiveBayesBigram, self).__init__()
        self.vocab_len = len(vocab)
        self.classes = num_labels
        self.p_class = np.zeros(self.classes)
        self.p_vocab = {y:{} for y in np.arange(self.classes)} # dict of dicts: second key is bigram tuple
        self.class_sum = np.zeros(self.classes) # NOT THE SAME AS P_CLASS
        for (x, y) in train_data:
            self.p_class[y] += 1
            for i in range(len(x)):
                if i+1 == len(x) or x[i+1] == 0: break # 0 padding
                if (x[i], x[i+1]) not in self.p_vocab[y]: 
                    self.p_vocab[y][(x[i], x[i+1])] = 0
                self.p_vocab[y][(x[i], x[i+1])] += 1
                self.class_sum[y] += 1
        for y, d in self.p_vocab.items():
            for bigram in d.keys():
                d[bigram] = (d[bigram] + alpha) / (self.class_sum[y] + self.vocab_len**2 * alpha)
        self.p_class /= np.sum(self.p_class)
        self.no_show = alpha * np.ones(self.classes) / (self.class_sum + self.vocab_len**2 * alpha)
        
    def forward(self, src):
        log_probs = np.log(self.p_class)
        for i in range(len(src)):
            if i+1 == len(src) or src[i+1] == 0: break # 0 padding
            bigram = (src[i], src[i+1])
            scores = [self.p_vocab[y][bigram] if bigram in self.p_vocab[y] else self.no_show[y] \
                      for y in range(self.classes)]
            log_probs += np.log(scores)
        return np.argmax(log_probs)

In [36]:
run_model_on_dataset(NaiveBayesBigram, wiki_data)

evaluate model, alpha =  0.01
accuracy:  0.9006
class-wise accuracies:  [0.9979511268802159, 0.1777365491651206]
evaluate model, alpha =  0.05
accuracy:  0.898
class-wise accuracies:  [0.9987007146069662, 0.150278293135436]
evaluate model, alpha =  0.1
accuracy:  0.8969
class-wise accuracies:  [0.999050522212783, 0.13803339517625232]
evaluate model, alpha =  0.5
accuracy:  0.893
class-wise accuracies:  [0.9994503023337165, 0.10241187384044527]
evaluate model, alpha =  1
accuracy:  0.8917
class-wise accuracies:  [0.9996002198790666, 0.09053803339517626]
evaluate model, alpha =  3
accuracy:  0.8894
class-wise accuracies:  [0.9998001099395333, 0.06975881261595547]
evaluate model, alpha =  6
accuracy:  0.888
class-wise accuracies:  [0.9998001099395333, 0.0575139146567718]
evaluate model, alpha =  10
accuracy:  0.8872
class-wise accuracies:  [0.9998500824546499, 0.05083487940630798]
best model alpha=0.01, evaluating on final test dataset
accuracy:  0.8887
class-wise accuracies:  [0.99990016

0.8886637893067911

In [37]:
run_model_on_dataset(NaiveBayesBigram, fake_data)

evaluate model, alpha =  0.01
accuracy:  0.8056
class-wise accuracies:  [0.9983379501385041, 0.16137931034482758, 0.04861111111111111, 0.32904734073641145]
evaluate model, alpha =  0.05
accuracy:  0.8018
class-wise accuracies:  [0.9983379501385041, 0.13655172413793104, 0.041666666666666664, 0.3185271770894214]
evaluate model, alpha =  0.1
accuracy:  0.7984
class-wise accuracies:  [0.9983379501385041, 0.11310344827586206, 0.027777777777777776, 0.309760374050263]
evaluate model, alpha =  0.5
accuracy:  0.7813
class-wise accuracies:  [0.9986149584487535, 0.041379310344827586, 0.013888888888888888, 0.24254821741671537]
evaluate model, alpha =  1
accuracy:  0.7735
class-wise accuracies:  [0.9986149584487535, 0.019310344827586208, 0.0, 0.20806545879602573]
evaluate model, alpha =  3
accuracy:  0.7573
class-wise accuracies:  [0.9993074792243767, 0.001379310344827586, 0.0, 0.12039742840444184]
evaluate model, alpha =  6
accuracy:  0.7481
class-wise accuracies:  [1.0, 0.001379310344827586, 0.0,

0.7431632653061224

In [38]:
print_f1_scores(NaiveBayesBigram, wiki_data, alpha=0.01, multi_label=False) # best alpha=0.5 (see above)

label 'not attack':
f1 score: 0.992429, precision: 0.9852413524930625, recall: 0.999722
label 'attack':
f1 score: 0.288582, precision: 0.9175531914893617, recall: 0.171216


In [39]:
print_f1_scores(NaiveBayesBigram, fake_data, alpha=0.01, multi_label=True) # best alpha=0.5 (see above)

label 'unrelated':
f1 score: 0.985487, precision: 0.9714687396625868, recall: 0.999915
label 'agree':
f1 score: 0.244477, precision: 0.7155172413793104, recall: 0.147425
label 'disagree':
f1 score: 0.013986, precision:   0.5, recall: 0.007092
label 'discuss':
f1 score: 0.443344, precision: 0.7840466926070039, recall: 0.309049


In [40]:
np.mean([0.985487, 0.244477, 0.013986, 0.443344])

0.4218235

In [ ]:
# OLD NAIVE BAYES - VERY BAD, DONT USE (runs out of memory)
"""
# slightly more sophisticated model
class NaiveBayesBigram(nn.Module):
    def __init__(self, vocab, num_labels, train_data, alpha=0.001):
        super(NaiveBayesBigram, self).__init__()
        self.vocab_len = len(vocab)
        self.classes = num_labels
        self.p_class = np.zeros(self.classes)
        self.p_vocab = alpha * np.ones((self.classes, self.vocab_len, self.vocab_len))
        print("start training")
        for (x, y) in train_data:
            self.p_class[y] += 1
            for i in range(len(x)):
                if i == 0: continue # skip first one
                if (x[i+1] == 0): break # 0 padding
                self.p_vocab[y, x[i], x[i+1]] += 1
        print("done with training")
        self.p_class /= np.sum(self.p_class)
        for i in range(self.p_vocab.shape[0]):
            self.p_vocab[i] = self.p_vocab[i] / np.sum(self.p_vocab[i])
        
    def forward(self, src):
        log_probs = np.log(self.p_class)
        for i in range(len(src)):
            if (i == 0): continue
            if (src[i+1] == 0): break
            log_probs += np.log(self.p_vocab[:, src[i], src[i+1]])
        return np.argmax(log_probs)
"""

# Data analysis

In [33]:
import pandas as pd
# wiki dataset
cutoff = 0.3
comment_df = pd.read_csv("../data/attack_annotated_comments.tsv", sep ='\t')
comment_df["comment"] = comment_df["comment"].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
comment_df["comment"] = comment_df["comment"].apply(lambda x: x.replace("TAB_TOKEN", " "))
annotation_df = pd.read_csv("../data/attack_annotations.tsv",  sep='\t')
annotation_df = (annotation_df.groupby("rev_id")["attack"].mean() > cutoff)
annotation_df = annotation_df.to_frame().reset_index()
final_df = pd.merge(comment_df, annotation_df, how='inner', on=['rev_id'])

In [34]:
for i, row in enumerate(final_df[final_df['attack'] == True]['comment']):
    if len(row) < 100:
        print(row)
        print()
    if i > 10: break

  Iraq is not good  ===  ===  USA is bad   

Anon  :What the heck are you talking about? This is an encyclopedia, not a book store. 

i have a dick, its bigger than yours! hahaha



In [35]:
for i, row in enumerate(final_df[final_df['attack'] == False]['comment']):
    if len(row) < 100:
        print(row)
        print()
    if i > 10: break

This page will need disambiguation. 



In [36]:
# fake news dataset
body_df = pd.read_csv("../data/fake_news_bodies.csv")
stance_df = pd.read_csv("../data/fake_news_stances.csv")

In [38]:
print(len(stance_df))
print(len(comment_df))

49972
115864
